<a href="https://colab.research.google.com/github/yashugupta786/word_vec_similarity/blob/master/bigram_word_vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [3]:
df = pd.read_csv('/content/simpsons_dataset.csv')
df.shape

(158314, 2)

In [4]:
df = df.dropna().reset_index(drop=True)
df.isnull().sum()

raw_character_text    0
spoken_words          0
dtype: int64

In [0]:
nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [0]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['spoken_words'])

In [7]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.96 mins


In [8]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(85964, 1)

In [9]:
df_clean.head()

,clean
0,actually little disease magazine news show nat...
2,know sure like talk touch lesson plan teach
3,life worth live
4,poll open end recess case decide thought final...
7,victory party slide


In [10]:
from gensim.models.phrases import Phrases, Phraser

INFO - 08:01:43: 'pattern' package not found; tag filters are not available for English


In [0]:
sent = [row.split() for row in df_clean['clean']]

In [12]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 08:02:09: collecting all words and their counts
INFO - 08:02:09: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 08:02:09: PROGRESS: at sentence #10000, processed 63561 words and 52816 word types
INFO - 08:02:10: PROGRESS: at sentence #20000, processed 130943 words and 99866 word types
INFO - 08:02:10: PROGRESS: at sentence #30000, processed 192972 words and 138532 word types
INFO - 08:02:10: PROGRESS: at sentence #40000, processed 249842 words and 172659 word types
INFO - 08:02:10: PROGRESS: at sentence #50000, processed 311265 words and 208566 word types
INFO - 08:02:10: PROGRESS: at sentence #60000, processed 373588 words and 243702 word types
INFO - 08:02:10: PROGRESS: at sentence #70000, processed 436441 words and 278740 word types
INFO - 08:02:10: PROGRESS: at sentence #80000, processed 497829 words and 311886 word types
INFO - 08:02:10: collected 330804 word types from a corpus of 537160 words (unigram + bigrams) and 85964 sentences
INFO - 08:02:10: us

In [14]:
bigram = Phraser(phrases)


INFO - 08:02:55: source_vocab length 330804
INFO - 08:02:58: Phraser built with 126 phrasegrams


In [0]:
sentences = bigram[sent]


In [0]:
sentences
for i in sentences:
  print(i)

In [22]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

30178

In [23]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['oh', 'like', 'know', 'get', 'hey', 'think', 'right', 'look', 'want', 'come']

In [0]:
import multiprocessing

from gensim.models import Word2Vec

In [0]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                  )

In [27]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 08:09:41: collecting all words and their counts
INFO - 08:09:41: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 08:09:41: PROGRESS: at sentence #10000, processed 61718 words, keeping 9558 word types
INFO - 08:09:41: PROGRESS: at sentence #20000, processed 127351 words, keeping 14506 word types
INFO - 08:09:41: PROGRESS: at sentence #30000, processed 187829 words, keeping 17619 word types
INFO - 08:09:42: PROGRESS: at sentence #40000, processed 243332 words, keeping 20385 word types
INFO - 08:09:42: PROGRESS: at sentence #50000, processed 303182 words, keeping 22878 word types
INFO - 08:09:42: PROGRESS: at sentence #60000, processed 363940 words, keeping 25200 word types
INFO - 08:09:42: PROGRESS: at sentence #70000, processed 425408 words, keeping 27401 word types
INFO - 08:09:42: PROGRESS: at sentence #80000, processed 485464 words, keeping 29275 word types
INFO - 08:09:42: collected 30178 word types from a corpus of 523700 raw words and 85964 sentence

Time to build vocab: 0.04 mins


In [28]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 08:10:03: training model with 3 workers on 3319 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 08:10:04: EPOCH 1 - PROGRESS: at 37.40% examples, 74816 words/s, in_qsize 0, out_qsize 0
INFO - 08:10:05: EPOCH 1 - PROGRESS: at 78.53% examples, 76880 words/s, in_qsize 0, out_qsize 0
INFO - 08:10:06: worker thread finished; awaiting finish of 2 more threads
INFO - 08:10:06: worker thread finished; awaiting finish of 1 more threads
INFO - 08:10:06: worker thread finished; awaiting finish of 0 more threads
INFO - 08:10:06: EPOCH - 1 : training on 523700 raw words (198820 effective words) took 2.6s, 77786 effective words/s
INFO - 08:10:07: EPOCH 2 - PROGRESS: at 37.40% examples, 75235 words/s, in_qsize 0, out_qsize 0
INFO - 08:10:08: EPOCH 2 - PROGRESS: at 74.71% examples, 72498 words/s, in_qsize 0, out_qsize 0
INFO - 08:10:09: worker thread finished; awaiting finish of 2 more threads
INFO - 08:10:09: worker thread finished; awaiting finish of 1 mo

Time to train the model: 1.33 mins


In [29]:
w2v_model.init_sims(replace=True)

INFO - 08:12:43: precomputing L2-norms of word weight vectors


In [30]:
w2v_model.wv.most_similar(positive=["homer_simpson"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('pleased', 0.7633680701255798),
 ('congratulation', 0.7631620168685913),
 ('recent', 0.7597476243972778),
 ('governor', 0.7590283155441284),
 ('easily', 0.7576010823249817),
 ('council', 0.7408592700958252),
 ('hutz', 0.7347520589828491),
 ('robert', 0.7318389415740967),
 ('simon', 0.7308964729309082),
 ('committee', 0.7258784770965576)]

In [31]:
w2v_model.wv.most_similar(positive=["homer"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('marge', 0.7794296145439148),
 ('rude', 0.7755630016326904),
 ('bongo', 0.7644156813621521),
 ('snuggle', 0.7563874125480652),
 ('wife', 0.7388522624969482),
 ('gee', 0.7364005446434021),
 ('sorry', 0.7309480309486389),
 ('worry', 0.727098822593689),
 ('hammock', 0.7240004539489746),
 ('sweetheart', 0.7224789261817932)]

In [32]:
w2v_model.wv.most_similar(positive=["dr_hibbert"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('convince', 0.8436061143875122),
 ('rabbi', 0.831490159034729),
 ('hearing', 0.8250008821487427),
 ('catholic', 0.7999840378761292),
 ('bitch', 0.7841911911964417),
 ('rude', 0.7727895379066467),
 ('attract', 0.7696405649185181),
 ('stress', 0.7687424421310425),
 ('technically', 0.7678347229957581),
 ('crisis', 0.7677313089370728)]

In [33]:
w2v_model.wv.most_similar(positive=["bart"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lisa', 0.8431563377380371),
 ('surprised', 0.7916699051856995),
 ('homework', 0.7898970246315002),
 ('mom', 0.7895394563674927),
 ('convince', 0.7756944894790649),
 ('upset', 0.7745141386985779),
 ('mom_dad', 0.77126145362854),
 ('strangle', 0.7669622898101807),
 ('substitute', 0.7586274147033691),
 ('typical', 0.7566486597061157)]

In [37]:
from gensim.models import Phrases
documents = ["the mayor of new york was there", "machine learning can be useful sometimes","new york mayor was present","My name is yashu gupta"]

sentence_stream = [doc.split(" ") for doc in documents]
bigram = Phrases(sentence_stream, min_count=1, threshold=2)
sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']
print(bigram[sent])

INFO - 08:40:58: collecting all words and their counts
INFO - 08:40:58: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 08:40:58: collected 37 word types from a corpus of 23 words (unigram + bigrams) and 4 sentences
INFO - 08:40:58: using 37 counts as vocab in Phrases<0 vocab, min_count=1, threshold=2, max_vocab_size=40000000>


['the', 'mayor', 'of', 'new_york', 'was', 'there']


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

from nltk.corpus import stopwords

from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

%matplotlib inline

In [0]:
data = pd.read_csv('/content/hotel-reviews.csv')

In [41]:
stopWords = stopwords.words('english')

# pre processing data
def cleanData(sentence):
    processedList = ""
    
    # convert to lowercase, ignore all special characters - keep only alpha-numericals and spaces (not removing full-stop here)
    sentence = re.sub(r'[^A-Za-z0-9\s.]',r'',str(sentence).lower())
    sentence = re.sub(r'\n',r' ',sentence)
    
    # remove stop words
    sentence = " ".join([word for word in sentence.split() if word not in stopWords])
    
    return sentence

LookupError: ignored

In [44]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stopWords = stopwords.words('english')

# pre processing data
def cleanData(sentence):
    processedList = ""
    
    # convert to lowercase, ignore all special characters - keep only alpha-numericals and spaces (not removing full-stop here)
    sentence = re.sub(r'[^A-Za-z0-9\s.]',r'',str(sentence).lower())
    sentence = re.sub(r'\n',r' ',sentence)
    
    # remove stop words
    sentence = " ".join([word for word in sentence.split() if word not in stopWords])
    
    return sentence

In [46]:
cleanData(data['Description'][2])

'booked hotel hotwire lowest price could find. got front desk manager gave us smoking room. argued little baby would booked room known smoking. manager would hear anything told hotwire books cheapest rooms available. get go unhappy. great deal persuasion discussion got nonsmoking room. thereafter room minimal amenities. besides great location near dupont circle much say overpriced hotel. room small ok condition. bathroom small tub. bathroom amenities also minimal. fridge microwave rent fridge staff keep baby things. parking costs per day best drive here. also breakfast included. lobby small feels old. thing available coffee lobby decent. poor service minimal amenities small rooms small bathrooms view great location. distance metro either mcpherson station dupont station. try look better available.'

In [0]:
data['Description'] = data['Description'].map(lambda x: cleanData(x))

In [0]:
tmp_corpus = data['Description'].map(lambda x: x.split('.'))

In [49]:
# corpus [[w1,w2,w3..],[..]]
corpus = []
for i in tqdm(range(len(tmp_corpus))):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

100%|██████████| 38932/38932 [00:02<00:00, 18997.45it/s]


In [50]:
num_of_sentences = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

Num of sentences - 444271
Num of words - 3111121


In [51]:
phrases = Phrases(sentences=corpus,min_count=25,threshold=50)
bigram = Phraser(phrases)

INFO - 08:58:49: collecting all words and their counts
INFO - 08:58:49: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 08:58:49: PROGRESS: at sentence #10000, processed 70801 words and 55905 word types
INFO - 08:58:49: PROGRESS: at sentence #20000, processed 141274 words and 98827 word types
INFO - 08:58:49: PROGRESS: at sentence #30000, processed 212516 words and 137942 word types
INFO - 08:58:49: PROGRESS: at sentence #40000, processed 282042 words and 173306 word types
INFO - 08:58:49: PROGRESS: at sentence #50000, processed 351277 words and 206238 word types
INFO - 08:58:49: PROGRESS: at sentence #60000, processed 421236 words and 238626 word types
INFO - 08:58:49: PROGRESS: at sentence #70000, processed 491663 words and 270170 word types
INFO - 08:58:50: PROGRESS: at sentence #80000, processed 563363 words and 301134 word types
INFO - 08:58:50: PROGRESS: at sentence #90000, processed 632424 words and 329645 word types
INFO - 08:58:50: PROGRESS: at sentence #10

In [0]:
for index,sentence in enumerate(corpus):
    corpus[index] = bigram[sentence]

In [0]:
# shuffle corpus
def shuffle_corpus(sentences):
    shuffled = list(sentences)
    random.shuffle(shuffled)
    return shuffled

In [54]:
# sg - skip gram |  window = size of the window | size = vector dimension
size = 100
window_size = 2 # sentences weren't too long, so
epochs = 100
min_count = 2
workers = 4

# train word2vec model using gensim
model = Word2Vec(corpus, sg=1,window=window_size,size=size,
                 min_count=min_count,workers=workers,iter=epochs,sample=0.01)

INFO - 08:59:38: collecting all words and their counts
INFO - 08:59:38: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 08:59:38: PROGRESS: at sentence #10000, processed 68251 words, keeping 8812 word types
INFO - 08:59:38: PROGRESS: at sentence #20000, processed 136136 words, keeping 12451 word types
INFO - 08:59:38: PROGRESS: at sentence #30000, processed 204641 words, keeping 15230 word types
INFO - 08:59:38: PROGRESS: at sentence #40000, processed 271576 words, keeping 17587 word types
INFO - 08:59:38: PROGRESS: at sentence #50000, processed 338157 words, keeping 19539 word types
INFO - 08:59:38: PROGRESS: at sentence #60000, processed 405603 words, keeping 21363 word types
INFO - 08:59:38: PROGRESS: at sentence #70000, processed 473392 words, keeping 23191 word types
INFO - 08:59:38: PROGRESS: at sentence #80000, processed 542497 words, keeping 24845 word types
INFO - 08:59:38: PROGRESS: at sentence #90000, processed 608985 words, keeping 26270 word types


In [55]:
model.most_similar('boston')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
INFO - 09:12:32: precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('seattle', 0.7688531875610352),
 ('san_francisco', 0.7484472393989563),
 ('philly', 0.7469840049743652),
 ('denver', 0.7364922761917114),
 ('sf', 0.7299060225486755),
 ('chicago', 0.7267497777938843),
 ('dc', 0.7259232997894287),
 ('san_diego', 0.718490719795227),
 ('san_fran', 0.7101852893829346),
 ('dallas', 0.7021840214729309)]

In [56]:
model.most_similar('yashu_gupta')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

In [57]:
model.most_similar("ramada")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('drury', 0.6127438545227051),
 ('holiday_inn', 0.6068820953369141),
 ('wyndham', 0.554410457611084),
 ('hilton_garden', 0.5532642602920532),
 ('la_quinta', 0.5508935451507568),
 ('bigname', 0.5388356447219849),
 ('motel', 0.5385938882827759),
 ('best_western', 0.5372475385665894),
 ('brookwood', 0.5355062484741211),
 ('tarzana', 0.5352388620376587)]

In [58]:
model.most_similar("taj mahal")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

In [59]:
model.most_similar('taj_mahal')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

In [60]:
model.most_similar('india')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lombard_street', 0.48105624318122864),
 ('mmmmm', 0.4479011297225952),
 ('coppolas', 0.4465385377407074),
 ('nightit', 0.4442659020423889),
 ('chineese', 0.43816423416137695),
 ('motorway', 0.43526691198349),
 ('cloud', 0.434867799282074),
 ('trailhead', 0.4317193329334259),
 ('italian', 0.43124300241470337),
 ('filbert', 0.42906200885772705)]

In [61]:
model.most_similar('chinese')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('indian', 0.5720556974411011),
 ('kjs', 0.5580602884292603),
 ('beccos', 0.5493707060813904),
 ('italian', 0.5345851182937622),
 ('homecooked', 0.5315892100334167),
 ('thai', 0.5280975103378296),
 ('wollensky', 0.5228995680809021),
 ('berghoffs', 0.5208449363708496),
 ('gallaghers', 0.5193091630935669),
 ('tasca', 0.5167564153671265)]

ERROR! Session/line number was not unique in database. History logging moved to new session 59
